This notebook is an example of what happens when someone with very little knowledge or experience in probabilistic programming languages, looks at some tutorials, gets some data, and cut'n'pastes things together and shuffles them around until something runs.

This is certainly wrong, but can hopefully provide a baseline for a working model. 

In [14]:
import os

import jax.numpy as jnp
import jax.random as random
from jax.scipy.special import logsumexp

import numpyro
import numpyro.distributions as dist

from numpyro.infer import MCMC, NUTS, Predictive



Get a sample of player data.  Just look at forwards, with data up to the middle of the 2019/20 season.

In [2]:
from airsenal.framework.prediction_utils import *

In [3]:
data, names = process_player_data("FWD","1920",20)

Filling history dataframe for Sergio Agüero: 0/80 done
Filling history dataframe for Harry Kane: 1/80 done
Filling history dataframe for Pierre-Emerick Aubameyang: 2/80 done
Filling history dataframe for Jamie Vardy: 3/80 done
Filling history dataframe for Gabriel Fernando de Jesus: 4/80 done
Filling history dataframe for Alexandre Lacazette: 5/80 done
Filling history dataframe for Roberto Firmino: 6/80 done
Filling history dataframe for Marcus Rashford: 7/80 done
Filling history dataframe for Romelu Lukaku: 8/80 done
Filling history dataframe for Tammy Abraham: 9/80 done
Filling history dataframe for Raúl Jiménez: 10/80 done
Filling history dataframe for Callum Wilson: 11/80 done
Filling history dataframe for Sébastien Haller: 12/80 done
Filling history dataframe for Marko Arnautovic: 13/80 done
Filling history dataframe for Teemu Pukki: 14/80 done
Filling history dataframe for Danny Ings: 15/80 done
Filling history dataframe for Olivier Giroud: 16/80 done
Filling history dataframe fo

this gives us "names", which is actually a list of player_ids, and the "data" is a dict with keys "nplayer" (int), "nmatch" (int), "minutes" (numpy array with shape (nplayer, nmatch)), "y" (numpy array with shape (nplayer, nmatch, 3)), and "alpha" (numpy array with shape (3,)).


### numpyro model

The below is certainly wrong.   I think "plate" broadcasts over all elements in an array.   Probably we don't want to broadcast over both players and matches, but rather just do it over players.

The DirichletMultinomial wants a "concentration" parameter called alpha, which is presumably what the "alpha" we have in our data is, but ought this to be normalised?



In [19]:
def model(nplayer, nmatch, minutes, y, alpha):


    with numpyro.plate("nmatch", nmatch):
        with numpyro.plate("nplayer", nplayer):
            return numpyro.sample("obs", dist.DirichletMultinomial(concentration=alpha), obs=y)

In [6]:
kernel = NUTS(model)
mcmc = MCMC(
    kernel,
    num_warmup=1500,
    num_samples=3000,
    num_chains=1,
    progress_bar=True
)

In [7]:
rng_key, rng_key_predict = random.split(random.PRNGKey(44))
mcmc.run(rng_key,
         data["nplayer"],
         data["nmatch"],
         data["minutes"],
         data["y"],
         data["alpha"]
)

sample: 100%|██████████████████████████████████| 4500/4500 [02:24<00:00, 31.12it/s, 9 steps of size 8.32e-03. acc. prob=0.81]


In [11]:
samples = mcmc.get_samples()

In [12]:
predictor = Predictive(model, posterior_samples=samples)
predictions = predictor(rng_key_predict,
                        data["nplayer"],
                        data["nmatch"],
                        data["minutes"],
                        data["y"],
                        data["alpha"]
          )

In [13]:
predictions["obs"]

DeviceArray([[[[2, 0, 1],
               [1, 0, 1],
               [2, 1, 2],
               ...,
               [0, 0, 2],
               [0, 0, 2],
               [1, 0, 0]],

              [[0, 0, 1],
               [0, 0, 3],
               [0, 0, 1],
               ...,
               [0, 1, 1],
               [1, 0, 1],
               [0, 0, 2]],

              [[0, 0, 0],
               [0, 0, 0],
               [0, 0, 0],
               ...,
               [0, 0, 2],
               [0, 0, 1],
               [0, 0, 4]],

              ...,

              [[0, 0, 0],
               [0, 0, 0],
               [0, 1, 0],
               ...,
               [0, 0, 3],
               [1, 0, 0],
               [0, 0, 0]],

              [[0, 0, 0],
               [0, 0, 0],
               [0, 0, 1],
               ...,
               [0, 0, 3],
               [0, 0, 4],
               [0, 0, 0]],

              [[0, 0, 0],
               [0, 0, 0],
               [0, 0, 0],
            